In [ ]:
import sys
import numpy as np
from random import randint
import random
import math
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import operator
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as GNB
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from deap import creator, base, tools, algorithms
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/sukrverm/Documents/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
file1 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/ogling_data/train.csv'
df = pd.read_csv(file1)
print(df.head())

corpus = list(df["Description"])
target = list(df["Category"])


In [ ]:
infinity = float("inf") #sentinel maximum value
m = 70 #no of bats (50-70)
T = 100 # no of iterations(100)

bat_pos = defaultdict(list)
bat_vel = defaultdict(list)
bat_loudness = []
bat_rate = []
bat_fitness = []
bat_frequency = []

global_bat_pos = []
init_rate_vector = []
feature_vector = []

#hyper parameters
alpha = 0.9
gamma = 0.9
epsilon = random.uniform(-1,1)
fmin=0
fmax =1

In [ ]:
#do once
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words=stop_words)
X = vectorizer.fit_transform(corpus)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print(X.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))

X_train = X
Y_train = target

file2 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/ogling_data/dev.csv'
df_dev = pd.read_csv(file2)
print(df_dev.head())
corpus_dev = list(df_dev["Description"])
target_dev = list(df_dev["Category"])
X_dev = vectorizer.transform(corpus_dev)
print(X_dev.shape)
Y_dev = target_dev


In [ ]:
def buildWordVector(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    tokens = tokenize(tweet)
    print(tokens)
    count = 0.
    for word in tokens:
        try:
            vec += model.word_vec(word).reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
def tokenize(tweet):
    try:
        tweet = unicode(tweet.decode('utf-8').lower())
        tokens = tokenizer.tokenize(tweet)
        tokens = filter(lambda t: not t.startswith('@'), tokens)
        tokens = filter(lambda t: not t.startswith('#'), tokens)
        tokens = filter(lambda t: not t.startswith('http'), tokens)
        return tokens
    except:
        print('NC')
        return 'NC'

In [ ]:
corpus_train_vecs_w2v = np.concatenate([buildWordVector(tweet, 300) for tweet in corpus])
corpus_dev_vecs_w2v = np.concatenate([buildWordVector(tweet, 300) for tweet in corpus_dev])

In [ ]:
print(corpus_dev_vecs_w2v.shape)
print(corpus_dev_vecs_w2v[0])

In [ ]:
#auxillary  functions
def extract_subset(z,cols):
    return z[:,cols]

In [ ]:
def svm_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    #for j in range(0, X_train.shape[1]):
    for j in range(0, corpus_train_vecs_w2v.shape[1]):
        if x[j] == 1:
            cols.append(j)
    #z1_prime = extract_subset(X_train, cols)
    #z2_prime = extract_subset(X_dev, cols)
    z1_prime = extract_subset(corpus_train_vecs_w2v, cols)
    z2_prime = extract_subset(corpus_dev_vecs_w2v, cols)
    clf = svm.LinearSVC(C=1.0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_dev,y_pred)
    pre = sklearn.metrics.precision_score(Y_dev,y_pred)
    fscore = sklearn.metrics.f1_score(Y_dev,y_pred)
    recall = sklearn.metrics.recall_score(Y_dev,y_pred)
    return (acc),pre,fscore,recall



In [ ]:
def rf_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = RandomForestClassifier(n_jobs=2,random_state=0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def knn_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = KNN(3,weights='uniform')
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def nb_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = GNB()
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def initialise(m,n):
    for i in range(0,m):
        for j in range(0,n):
            x = randint(0,1)
            bat_pos[i].append(x)
            bat_vel[i].append(0)

        init_rate_vector.append(random.uniform(0, 1))

        bat_frequency.append(init_rate_vector[i])
        bat_loudness.append(random.uniform(1,2))
        bat_rate.append(init_rate_vector[i])
        bat_fitness.append(-infinity)

In [ ]:
def core():
    initialise(m,n)
    global_fit = -infinity
    for t in range (0,T):
        print("T:" , t)
        for i in range(0,m):
            accuracy,pre,fscore,recall = svm_calc_fitness(bat_pos[i])
            '''
            with open("dumpPerformanceBat_svm_kpi.txt", "a") as text_file:
                text_file.write("{} {} {} {} {}".format(accuracy,pre,fscore,recall,global_fit) + "\n")
            with open("dumpPerformanceBat_svm.txt", "a") as text_file:
                text_file.write("{} {} {} {}".format(t,accuracy,bat_pos[i].count(1),global_fit)+"\n")
            #print("t={} accuracy={} #features={} global_fit={}".format(t,accuracy,bat_pos[i].count(1),global_fit))
            '''
            acc = accuracy

            rand = randint(0,1)
            if rand < bat_loudness[i] and acc > bat_fitness[i]:
                bat_fitness[i]=acc
                bat_loudness[i]= alpha * bat_loudness[i]
                bat_rate[i]= init_rate_vector[i] * (1 - math.exp(-(gamma * t)))

        maxindex,maxfit = max(enumerate(bat_fitness), key=operator.itemgetter(1))

        if maxfit>global_fit :
            global_fit = maxfit
            global_bat_pos = bat_pos[maxindex]

        for i in range(0,m): # for each bat
            beta = random.uniform(0,1)
            rand = random.uniform(0,1)
            AvgA = sum(bat_loudness)/len(bat_loudness)

            if rand > bat_rate[i] :
                for j in range(0,n):
                    bat_pos[i][j] = bat_pos[i][j] + ( epsilon * AvgA )
                    sigma = randint(0,1)
                    if sigma < (1/(1+math.exp(-bat_pos[i][j]))) :
                        bat_pos[i][j]=1
                    else :
                        bat_pos[i][j]= 0

            rand = random.uniform(0,1)
            if rand < bat_loudness[i] and bat_fitness[i] < global_fit :
                for j in range (0,n):
                    bat_frequency[i] = fmin + (fmax-fmin)* beta
                    bat_vel[i][j] = bat_vel[i][j] + (global_bat_pos[j]-bat_pos[i][j]) * bat_frequency[i]
                    bat_pos[i][j] = bat_pos[i][j] + bat_vel[i][j]
                    sigma = random.uniform(0,1)
                    if sigma < (1 / (1 + math.exp(-bat_pos[i][j]))) :
                        bat_pos[i][j]=1
                    else :
                        bat_pos[i][j]=0
        with open("dumpPerformanceBat_rf.txt", "a") as text_file:
            text_file.write("{} {} {} ".format(t,global_bat_pos.count(1),global_fit)+"\n")
        


    feature_vector = global_bat_pos
    return feature_vector

In [ ]:
def BAT_algo():
    feature_vector = core()
    return feature_vector

In [ ]:
f = BAT_algo()
 

In [ ]:
class GeneticAlgorithm():
    
    def __init__(self, X_train, X_test, Y_train, Y_test, number_bits_to_mutate = 25.0):
        
        
        self.toolbox = base.Toolbox()
        self.ind_mut_pb = float(number_bits_to_mutate / X_train.shape[1])
        self.NGEN = 100
        self.MUTPB = 0.2
        self.CXPB = 0.5
        self.POPULATION_SIZE = 70
        self.NUMBER_TO_SELECT_FROM_POP = 35
        self.init_pop = list()
        self.X_tr = X_train
        self.X_te = X_test
        self.Y_train = Y_train
        self.Y_test = Y_test
        
    
    def initialise_toolbox(self):
        print('now initializing toolbox GA1')
        # I want to maximize session overlap as well as length
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        # representing individual as a list
        creator.create("Individual", list, fitness=creator.FitnessMax)
        # individual will be a list of 0/1
        self.toolbox.register("attr_bool", random.randint, 0, 1)
        IND_DIMENSIONALITY = self.X_tr.shape[1]
        # create template for individual
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_bool, n=IND_DIMENSIONALITY)
        #self.toolbox.register("individual", tools.initIterate, creator.Individual, self.get_ind)
        # create template for population
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        print('toolbox initialised')
    
    
    def initialise_fitness_function(self):
        print('building and registering the fitness function')
        
        def fitness(individual):
            
            
            accuracy,pre,fscore,recall = svm_calc_fitness(individual)
            return fscore,
            
        
        self.toolbox.register("evaluate", lambda individual: fitness(individual))
        self.toolbox.register("mate", tools.cxOnePoint)
        self.toolbox.register("mutate", tools.mutFlipBit, indpb=self.MUTPB)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
        print('fitness function ready')
    
    def run_algorithm(self):
        
        self.initialise_toolbox()
        self.initialise_fitness_function()
        
        print('mutation probability was calculated at %f' % self.ind_mut_pb)
        print('running the algorithm')
        toolbox = self.toolbox
        # create a population of size n
        pop = toolbox.population(n=self.POPULATION_SIZE)
        
        
        for gen in range(self.NGEN):
            # Select the next generation individuals
            if gen%5 == 0:
                print(gen)
            offspring = toolbox.select(pop, len(pop))
            
            for ind in offspring:
                if ind.fitness.valid:
                    pass
            # Clone the selected individuals
            offspring = map(toolbox.clone, offspring)

            # Apply crossover on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.CXPB:
                    #toolbox.mate(child1, child2)
                    tools.cxUniform(child1, child2, 0.5)
                    del child1.fitness.values
                    del child2.fitness.values

            # Apply mutation on the offspring
            for mutant in offspring:
                if random.random() < self.MUTPB:
                    # print('now doing mutation')
                    toolbox.mutate(mutant)
                    del mutant.fitness.values

            # Evaluate the individuals with an invalid fitness
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            
            added = 0
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
                

            # The population is entirely replaced by the offspring
            pop[:] = offspring
            top = tools.selBest(pop, k=self.NUMBER_TO_SELECT_FROM_POP)
            
            # Dump best individual to file
            fittest = tools.selBest(pop, k=1)[0]
            count = 0
            for i in range(0, len(fittest)):
                if fittest[i] == 1:
                    count = count + 1 
            with open("GA_performance.txt", "a") as text_file:
                accuracy,prec,fscore,recall = svm_calc_fitness(fittest)
                text_file.write("{} {}".format(count,fittest.fitness.values[0])+"\n")
                print(count,accuracy,prec,fscore,recall,fittest.fitness.values[0])
        
        
        
        
        max_fitness = float("-inf")
        
        for individual in tools.selBest(pop, k=1):
            if individual.fitness.valid:
                if individual.fitness.values[0] > max_fitness:
                    max_fitness = individual.fitness.values[0]
                    fittest_individual = individual
                    
        count = 0
        for i in range(0, len(fittest_individual)):
            if fittest_individual[i] == 1:
                count = count + 1        
        
        print(count, max_fitness)
        return fittest_individual

In [ ]:
ga = GeneticAlgorithm(corpus_train_vecs_w2v, corpus_dev_vecs_w2v, Y_train, Y_dev)

In [ ]:
fittest_individual = ga.run_algorithm()

In [ ]:
file3 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/ogling_data/test.csv'
df_test = pd.read_csv(file3)
print(df_test.head())
corpus_test = list(df_test["Description"])
target_test = list(df_test["Category"])
X_test = vectorizer.transform(corpus_test)
print(X_test.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))
Y_test = target_test

In [ ]:
corpus_test_vecs_w2v = np.concatenate([buildWordVector(tweet, 300) for tweet in corpus_test])

In [ ]:
def svm_calc_final_metrics(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    print(X_test.shape[0])
    #for j in range(0, X_train.shape[1]):
    for j in range(0, corpus_train_vecs_w2v.shape[1]):
        if x[j] == 1:
            cols.append(j)
    #z1_prime = extract_subset(X_train, cols)
    #z2_prime = extract_subset(X_test, cols)
    z1_prime = extract_subset(corpus_train_vecs_w2v, cols)
    z2_prime = extract_subset(corpus_test_vecs_w2v, cols)
    clf = svm.LinearSVC(C=1.0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
accuracy,prec,fscore,recall = svm_calc_final_metrics(fittest_individual)
print(accuracy,prec,fscore,recall,fittest_individual.fitness.values[0])